# 07/31 train-rmse:0.242975;	 valid-rmse:0.249397

- **V** Number of characters in title
- **V** Number of characters in description
- **V** tfidf of description
- **V** tfidf of title

- Number of words
- Average Word Length
- Number of stopwords
- Number of special characters

- if the description is empty (replace with 'NA' string is fine. and then create another boolean feature)
- try new features to see if the result improved
- search for keywords in description of title. i.e. "new" or "old"
- empty description will be 0 after tfidf?
- check the predicted value, see if they fall into [0,1]

In [1]:
import pandas as pd
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import numpy as np

In [2]:
# load dataset
train = pd.read_csv('data/train.csv')
#test = pd.read_csv('data/test.csv')

In [3]:
train["item_id"].is_unique

True

In [4]:
train=train[["item_id","title","description","deal_probability"]]

In [5]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
train['char_len_title'] = train.title.apply(lambda x: len(str(x)))
train['char_len_desc'] = train.description.apply(lambda x: len(str(x)))

In [6]:
train.head()

,item_id,title,description,deal_probability,char_len_title,char_len_desc
0,b912c3c6a6ad,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",0.12789,21,58
1,2dac0150717d,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",0.00000,17,41
2,ba83aefab5dc,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",0.43177,14,99
3,02996f1dd2ea,Автокресло,Продам кресло от0-25кг,0.80323,10,22
4,7c90be56d2ab,"ВАЗ 2110, 2003",Все вопросы по телефону.,0.20797,14,24


In [7]:
import nltk
from nltk.corpus import stopwords
stopWords = stopwords.words('russian')

In [8]:
# need to change data type from object to unicode
train["description"].dtype

dtype('O')

# Morphological analyzer for Russian language 14:25

In [ ]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
retoken = re.compile(r'[\'\w\-]+')
def normalize(text):
    text = retoken.findall(text.lower()) # make all text lowercase
    text = [morph.parse(x)[0].normal_form for x in text] # morphological analysis
    return ' '.join(text)

train['title'] = train['title'].astype(str)
train['description'] = train['description'].astype(str)

train['title'] = train['title'].apply(normalize)
train['description'] = train['description'].apply(normalize)

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# tfidf for description
train['description']=train['description'].fillna('NA')
tfidf = TfidfVectorizer(max_features=100, stop_words = stopWords)
tfidf_train = np.array(tfidf.fit_transform(train['description'].values.astype('U')).todense(), dtype=np.float16)
for i in range(100):
    train['tfidf_des_' + str(i)] = tfidf_train[:, i]

# tfidf for title
train['title']=train['title'].fillna('NA')
tfidf_title_train = np.array(tfidf.fit_transform(train['title'].values.astype('U')).todense(), dtype=np.float16)
for i in range(100):
    train['tfidf_title_' + str(i)] = tfidf_title_train[:, i]

In [ ]:
train.head()

In [ ]:
# split train and test dataset
from sklearn.model_selection import train_test_split

y = train['deal_probability']
X = train.drop(['deal_probability','item_id','title','description'],axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=99)
##X_val = X_val.drop(['deal_probability'],axis=1)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
import xgboost as xgb

# grid search result
params = {'eta': 0.3,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 1400,  
          'max_depth': 0, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'reg:logistic', 
          'eval_metric': 'rmse', 
          'random_state': 99, 
          'silent': True}

tr_data = xgb.DMatrix(X_train, y_train)
va_data = xgb.DMatrix(X_val, y_val)
del X_train
del X_val
del y_train
del y_val
import gc
gc.collect()

watchlist = [(tr_data, 'train'), (va_data, 'valid')]

model = xgb.train(params, tr_data, 1000, watchlist, maximize=False, early_stopping_rounds = 25, verbose_eval=5)

In [ ]:
# generate result for test dataset

#X_te = xgb.DMatrix(X_te)
#y_pred = model.predict(X_te, ntree_limit=model.best_ntree_limit)
#sub = pd.read_csv('../input/sample_submission.csv')
#sub['deal_probability'] = y_pred
#sub['deal_probability'].clip(0.0, 1.0, inplace=True)
#sub.to_csv('xgb_with_mean_encode_and_nlp.csv', index=False)
#sub.head()

In [ ]:
# plotting 

#from xgboost import plot_importance
#import matplotlib.pyplot as plt
#plot_importance(model)
#plt.gcf().savefig('feature_importance_xgb.png')
